<a href="https://colab.research.google.com/github/SkyChen1009/ML-project/blob/main/Store%20Sales%20-%20Time%20Series%20Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle
import numpy as np
import pandas as pd
import xgboost as xg
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import HistGradientBoostingRegressor
from pickle import FALSE
%matplotlib inline

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c store-sales-time-series-forecasting

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [4]:
!unzip store-sales-time-series-forecasting

unzip:  cannot find or open store-sales-time-series-forecasting, store-sales-time-series-forecasting.zip or store-sales-time-series-forecasting.ZIP.


# Load datasets

In [6]:
train = pd.read_csv("../content/train.csv")           # Training data with sales information
test = pd.read_csv("../content/test.csv")            # Test data for predictions
stores = pd.read_csv("../content/stores.csv")          # Store-specific metadata
sub = pd.read_csv("../content/sample_submission.csv")      # Sample submission format
transactions = pd.read_csv("../content/transactions.csv")    # Transaction counts
oil = pd.read_csv("../content/oil.csv")             # Oil price data
holidays_events = pd.read_csv("../content/holidays_events.csv") # Holidays and event data

# Fill missing values in oil prices with the mean

In [7]:
oil_mean=oil['dcoilwtico'].mean()
oil['dcoilwtico'].fillna(oil_mean, inplace=True)

<ipython-input-7-3d495a20026e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oil['dcoilwtico'].fillna(oil_mean, inplace=True)


# Merge oil data with train and test sets on date

In [8]:
train=train.merge(oil, how='left', on='date')
train['dcoilwtico'].fillna(oil_mean, inplace=True)
test=test.merge(oil, how='left', on='date')
test['dcoilwtico'].fillna(oil_mean, inplace=True)

<ipython-input-8-14ec24b32ac7>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['dcoilwtico'].fillna(oil_mean, inplace=True)
<ipython-input-8-14ec24b32ac7>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

# Merge store data with train and test sets on store number

In [9]:
train=train.merge(stores, how="left", on="store_nbr")
train['store_type']=train['type']
train=train.drop("type", axis=1)    # Rename 'type' to 'store_type'
test=test.merge(stores, how="left", on="store_nbr")
test["store_type"]=test["type"]
test=test.drop("type", axis=1)

# Merge holidays and events data with train and test sets on date

In [10]:
train=train.merge(holidays_events, how="left", on="date")
train["type"].fillna("", inplace=True)
train["locale"].fillna("", inplace=True)
train["transferred"].fillna(False, inplace=True)
test=test.merge(holidays_events, how="left", on="date")
test["type"].fillna("", inplace=True)
test["locale"].fillna("", inplace=True)
test["transferred"].fillna(False, inplace=True)

<ipython-input-10-94d9ee222b3f>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["type"].fillna("", inplace=True)
<ipython-input-10-94d9ee222b3f>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.meth

# Encode categorical features using LabelEncoder

In [11]:
label_encoder=LabelEncoder()
object_columns=['date', 'family', 'city', 'state', 'type', 'cluster', "store_type", "locale", "transferred"]
for column in object_columns:
  combined_data=pd.concat([train[column], test[column]], axis=0)
  label_encoder.fit(combined_data)
  train[column]=label_encoder.transform(train[column])
  test[column]=label_encoder.transform(test[column])

# Drop unnecessary features from train and test sets

In [12]:
feature_to_drop=['id', "description", "locale_name"]
train=train.drop(feature_to_drop, axis=1)
test=test.drop(feature_to_drop, axis=1)

# Prepare features and target variable for model training

In [13]:
X=train.drop(['sales'], axis=1)     # Features
y=train['sales']             # Target variable (sales)
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Linear Regression model

In [14]:
model=LinearRegression()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
y_pred=np.array([max(i, 0) for i in y_pred])
#print(mean_squared_log_error(y_pred, y_test, squared=False))

# Train a Histogram-based Gradient Boosting Regressor model

In [15]:
HGBR_model=HistGradientBoostingRegressor(max_iter=3000)
HGBR_model.fit(X_train, y_train)
y_pred=HGBR_model.predict(X_test)
y_pred=np.array([max(i,0) for i in y_pred])
#print(mean_squared_log_error(y_pred, y_test, squared=False))

# Train an XGBoost Regressor model

In [16]:
XGB_model=xg.XGBRegressor(n_estimators=400, learning_rate=0.03, max_depth=12)
XGB_model.fit(X_train, y_train)
y_pred=XGB_model.predict(X_test)
y_pred=np.array([max(i,0) for i in y_pred])

# Predict on the test set and save to submission format

In [17]:
pred = XGB_model.predict(test)
pred=np.array([max(i,0) for i in pred])
sub['sales']=pred
sub.to_csv('/content/sub.csv', index=False, header=True)